In [1]:
# Import các thư viện cần thiết
import torch
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from huggingface_hub import notebook_login

In [2]:
# Đăng nhập vào Hugging Face
notebook_login()

In [3]:
# Bước 1: Load dataset từ Hugging Face
dataset = load_dataset("UngLong/Say_It_Right")

In [4]:
# Bước 2: Chuẩn bị tokenizer
model_checkpoint = "uitnlp/CafeBERT"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [5]:
# Hàm tokenization
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples["free_text"], padding="max_length", truncation=True, max_length=256)
    tokenized_inputs["labels"] = examples["binary_label"]  # Dùng binary_label làm labels
    return tokenized_inputs

# Áp dụng tokenization
dataset = dataset.map(tokenize_function, batched=True, remove_columns=["free_text", "label_id", "binary_label"])

Map:   0%|          | 0/23847 [00:00<?, ? examples/s]

Map:   0%|          | 0/2654 [00:00<?, ? examples/s]

Map:   0%|          | 0/6626 [00:00<?, ? examples/s]

In [6]:
# Chia tập train và validation
train_dataset = dataset["train"]
valid_dataset = dataset["validation"]

In [7]:
# Bước 3: Load mô hình CafeBERT
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at uitnlp/CafeBERT and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Bước 4: Thiết lập tham số fine-tuning
training_args = TrainingArguments(
    output_dir="./cafebert_finetuned",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    push_to_hub=True,  # Đẩy mô hình lên Hugging Face Hub
    hub_model_id="UngLong/cafebert-toxic-classifier",  # Chỉ định repo trên Hugging Face
    report_to="none"  # Thêm dòng này để tắt W&B logging
)

In [12]:
# Bước 5: Định nghĩa Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset
)

In [13]:
# Bước 6: Tiến hành fine-tuning
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.283900,0.263470
2,0.212300,0.257331


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


RuntimeError: [enforce fail at inline_container.cc:603] . unexpected pos 3021873408 vs 3021873296

In [15]:
trainer.model.save_pretrained("./emergency_checkpoint")
tokenizer.save_pretrained("./emergency_checkpoint")


OSError: [Errno 28] No space left on device: './emergency_checkpoint'

In [19]:
!mv /kaggle/working/cafebert_finetuned /cafebert_backup
!mv /kaggle/working/logs /cafebert_backup/


In [20]:
!ls -lh /kaggle/working/cafebert_finetuned


ls: cannot access '/kaggle/working/cafebert_finetuned': No such file or directory


In [24]:
!find / -type d -name "cafebert_backup"


find: ‘/proc/118/task/118/net’: Invalid argument
find: ‘/proc/118/net’: Invalid argument
find: ‘/proc/134/task/134/net’: Invalid argument
find: ‘/proc/134/net’: Invalid argument
/cafebert_backup


In [25]:
import torch

checkpoint_path = "/cafebert_backup/pytorch_model_latest.bin"
torch.save(model.state_dict(), checkpoint_path)
print(f"Model checkpoint saved to {checkpoint_path}")


Model checkpoint saved to /cafebert_backup/pytorch_model_latest.bin


In [26]:
import os
print(os.listdir("/cafebert_backup"))


['training_args.bin', 'config.json', 'checkpoint-2238', 'logs', 'pytorch_model_latest.bin', 'README.md', 'model.safetensors', 'checkpoint-746', 'checkpoint-1492']


In [14]:
# Bước 7: Đẩy mô hình lên Hugging Face
trainer.push_to_hub()

OSError: [Errno 28] No space left on device

In [ ]:
from huggingface_hub import upload_folder

upload_folder(
    folder_path="/cafebert_backup/",  # Đường dẫn đến thư mục chứa model
    repo_id="UngLong/cafebert-toxic-classifier",  # Repo của bạn trên Hugging Face
    repo_type="model"
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/hf_api.py:9246: UserWarning: Warnings while validating metadata in README.md:
- empty or missing yaml metadata in repo card
  warnings.warn(f"Warnings while validating metadata in README.md:\n{message}")


rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

Upload 11 LFS files:   0%|          | 0/11 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

...

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

events.out.tfevents.1742284351.9ae82e8fbe87.31.0:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

pytorch_model_latest.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/UngLong/cafebert-toxic-classifier/commit/16f755a725277357e2dc94b39604491ee4cfa183', commit_message='Upload folder using huggingface_hub', commit_description='', oid='16f755a725277357e2dc94b39604491ee4cfa183', pr_url=None, repo_url=RepoUrl('https://huggingface.co/UngLong/cafebert-toxic-classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='UngLong/cafebert-toxic-classifier'), pr_revision=None, pr_num=None)